# Capacity Building
## Prerequisites
- Some basic understanding of Python variables, data types, looping, conditionals and functions will be of benefit.
- Completion of  01-basic-model.ipynb

## Introduction
Once a suitable model structure is identified to describe infection disease dynamics, parameter values need to be inferred to describe the flows between the compartments. These flows determine how many individuals move into and out of the compartments per unit time. The model parameter values can be estimated through existing knowledge or through fitting the model to available data, or some combination. These parameter values will govern the rate of change in the compartments (e.g. the rate of change in the number of susceptibles or immune individuals). When estimating these rates, the average per capita time for an event (i.e. the reciprocal of rate at which the event occurs) is the key consideration. For example, the average life expectancy and mortality rate are related as,
<br><center>Average life expectancy = 1 / mortality rate.</center>
<br>The ODE system will be numerically solved behind the scenes using an ODE solver.

## Data inputs
### Imports

Let's import some modules. A module is a library of Python code that we can leverage to provide useful functionality.<br> These may be part of the standard Python library, or be external packages

In [ ]:
# Install the summer package
# Pip is Python's standard package manager

%pip install summerepi

In [ ]:
from datetime import datetime, timedelta
from matplotlib import pyplot as plt  # matplotlib is a common visualisation package for Python
import pandas as pd

from summer import CompartmentalModel
# This time, we're going to do some interactive plotting!
pd.options.plotting.backend = "plotly"

## Import Philippines data

In [ ]:
# The data import module lives in a file on AuTuMN github - download it for colab use
!wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/philippines/import_phl_data.py

In [ ]:
from import_phl_data import get_population_and_epi_data

# Shareable google drive links
PHL_DOH_LINK = "1fFKoNVan7PS6BpBr01nwByNTLLu6z1AA"  # sheet 05 daily report.
PHL_FASSSTER_LINK = "15eDyTjXng2Zh38DVhmeNy0nQSqOMlGj3" # Fassster google drive zip file.
initial_population, df = get_population_and_epi_data(PHL_DOH_LINK, PHL_FASSSTER_LINK) 
notifications_target = df[start_date: end_date]['cases']  # used as calibration target later

# We define a day zero for the analysis.
COVID_BASE_DATE = datetime(2019, 12, 31)

### Visualise notification data

In [ ]:
notifications_target.plot()

## Build a model

In [ ]:
start_date = datetime(2021,1,1)  # Define the start date
end_date = start_date + timedelta(days=300)  # Define the duration

# Integer representation of the start and end dates.
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date- COVID_BASE_DATE).days

## Structuring Model Code

We will wrap the model building code in a function. Any parameterised input can be passed in to this function so the model can be easily modified.

In [ ]:
# This is a very basic starting function that simply creates a CompartmentalModel
# and initialises it with the initial_population

# Note that we are not passing anything into this function yet;
# the start and end dates, and the population are 'captured' from the external scope
# ie the global program environment

def build_base_model() -> CompartmentalModel:
    model = CompartmentalModel(
        times=(start_date_int, end_date_int),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
        timestep=1.0,
        ref_date=COVID_BASE_DATE
    )

    model.set_initial_population(distribution={"S": initial_population - 100, "I": 100})
    return model

### Adding inter-compartmental flows 

Now, let's add some flows for people to transition between the compartments. These flows will define the dynamics of our infection. We will add:

- an infection flow from S to E (using frequency-dependent transmission)
- an exposed individual becomes infected E to I
- a recovery flow from I to R

In [ ]:
def build_model_with_hardcoded_flows() -> CompartmentalModel:
    
    # We start by calling our base model building function from above
    # This allows us to easily create variations, always building on the same base
    model = build_base_model()
    # Susceptible people can get infected.
    model.add_infection_frequency_flow(name="infection", contact_rate=0.3, source="S", dest="E")

    # Expose people transition to infected.
    model.add_transition_flow(name="progression", fractional_rate=1/10, source="E", dest="I")

    # Infectious people recover.
    model.add_transition_flow(name="recovery", fractional_rate=1/7, source="I", dest="R")

    # Importantly, we will also request an output for the 'progression' flow, and name this 'incidence'
    # This will be available after a model run using the get_derived_outputs_df() method

    model.request_output_for_flow("incidence", "progression")

    return model

In [ ]:
model_with_flows = build_model_with_hardcoded_flows() 
# Inspect the new flows, which we just added to the model.
model_with_flows._flows

## Writing a better build_model - parameterization

In the function above, you can see a lot of hardcoded constants inside the flow arguments

Let's extract these to a dictionary, and rewrite the function to take this dictionary
as an argument

In [ ]:
parameters = {
    "contact_rate": 0.3,
    "progression_rate": 1/10,
    "recovery_rate": 1/10
}

In [ ]:
def build_model_with_flows(parameters: dict) -> CompartmentalModel:
    
    # Call the base model as before
    # This base model does not take parameters, but have a think about how it might...
    model = build_base_model()
    # Susceptible people can get infected.
    # Note that we now look up the parameters dictionary instead of hardcoding a constant
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )

    # Expose people transition to infected.
    model.add_transition_flow(
        name="progression", 
        fractional_rate=parameters["progression_rate"], 
        source="E", 
        dest="I"
    )

    # Infectious people recover.
    model.add_transition_flow(name="recovery", fractional_rate=parameters["recovery_rate"], source="I", dest="R")

    # Importantly, we will also request an output for the 'progression' flow, and name this 'incidence'
    # This will be available after a model run using the get_derived_outputs_df() method

    model.request_output_for_flow("incidence", "progression")

    return model

In [ ]:
model = build_model_with_flows(parameters=parameters)

In [ ]:
# Iterate through the flows in the model
# As you can see, they have the values we passed in from the dictionary
for f in model._flows:
    print(f, f.param)

In [ ]:
# Actually, that's handy - let's make it a function so we can reuse it later

def inspect_flows(model: CompartmentalModel):
    for f in model._flows:
        print(f, f.param)

### Running the model

Now we can calculate the outputs for the model over the requested time period. 
The model calculates the compartment sizes by solving a system of differential equations (defined by the flows we just added) over the requested time period.

In [ ]:
model.run()

### Display the model outputs

The recommended way to access the model's results is via the get_outputs_df() method

In [ ]:
mm_outputs_df = model.get_outputs_df()
mm_outputs_df[["E","I","R"]].plot() # Don't plot the susceptible compartment because of y-axis scale

**Exercise: Modify the code in the last cell to show the susceptible individuals over time.**

### Accessing derived outputs

Derived outputs are accessed in much the same way as the raw compartment outputs, via the get_derived_outputs_df() method

**Question: Which flow control contributes the most to notifications? Would you increase it or decrease it?**

In [ ]:
mm_derived_df = model.get_derived_outputs_df()
mm_derived_df.plot()

**Exercise: Build a model with some different parameter values and plot its outputs**

In [ ]:
# Exercise : build a model with some different parameter values
new_parameters = "create a dictionary here"

# What arguments does this need?
new_params_model = build_model_with_flows()

# Inspect the flows for this model
# Enter your code here...

# Plot the compartment sizes over time

# Plot incidence over time


## Time varying parameters (transition flow)
The rate at which people transition can be set as a constant, or it can be defined as a function of time. This is the case of all of the flows: every parameter can be a constant or a function of time. Parameters also take a ‘computed_values’ argument, which is a dictionary of values computed at runtime that is not specific to any individual flow.

In [ ]:
# We define this function globally - 
# it will be used as an argument later when we create flows in the model

def recovery_rate(time, computed_values):
    """
    Returns the recovery rate for a given time.
    People recover faster from June 2021 due to a magic drug.
    """
    if time < 518: # before treatment use
        return 0.1
    else:  # after treatment use
        return 0.4

In [ ]:
parameters_with_treatment = {
    "contact_rate": .3,
    "progression_rate": 1/10,
    "recovery_rate": recovery_rate
}
treatment_model = build_model_with_flows(parameters_with_treatment)

### Plot the outputs of the model with time-variant recovery rate

In [ ]:
treatment_model.run()
incidence_with_treatment = treatment_model.get_derived_outputs_df()["incidence"]
incidence_with_treatment.plot()

In [ ]:
incidence_without_treatment = model.get_derived_outputs_df()["incidence"]
incidence_without_treatment.plot()

In [ ]:
# That's probably a little hard to see - let's write a function we can reuse
# for model comparison

def plot_comparison(label_a: str, data_a: pd.Series, label_b: str, data_b: pd.Series, title: str):
    # Visualize the results.
    subplot = {"title": title, "xlabel": "Days", "ylabel": "Value"}
    fig, ax = plt.subplots(1, 1, figsize=(10,5), subplot_kw=subplot)


    ax.plot(data_a.index, data_a) 
    ax.plot(data_b.index, data_b)

    ax.legend([label_a, label_b])

In [ ]:
plot_comparison("no treatment", incidence_without_treatment, "with treatment", incidence_with_treatment, "Treatment effect on incidence")

# Can we capture the data with our model?

In [ ]:
plot_comparison("Notification", 
                notifications_target, 
                "Modelled",
                incidence_without_treatment,
                "Modelled vs data")

### Let's account for case detection

In [ ]:
prop_of_incidence_detected = .01
plot_comparison("Notification", 
                notifications_target, 
                "Modelled",
                incidence_without_treatment * prop_of_incidence_detected,
                "Modelled vs data")

### Now let's try to calibrate our model to the first epidemic wave

In [ ]:
# This cell pulls all of the above together; 
# Try changing the various values

calibrated_parameters = {
    "contact_rate": 0.3, 
    "progression_rate": 0.1,
    "recovery_rate": 0.1
}

calibrated_model = build_model_with_flows(calibrated_parameters)
calibrated_model.run()
calibrated_derived_outputs= calibrated_model.get_derived_outputs_df()

prop_of_incidence_detected = 0.01
plot_comparison("Notification", 
                notifications_target, 
                "Modelled",
                calibrated_derived_outputs["incidence"] * prop_of_incidence_detected,
                "Modelled vs data")

Reasonable fit to first wave obtained with the following parameters:

calibrated_parameters = {
    "contact_rate": 0.5, 
    "progression_rate": 0.1,
    "recovery_rate": .1
}

prop_of_incidence_detected = 0.02


## Summary

That's it for now, now you know how to:

- Define compartmental flows
- Understand the different flow types in summer
- Build reusable model code with a basic expression of parameters

A detailed API reference for the flow types can be found [here](http://summerepi.com/api/flows.html)
